In [1]:
import pandas as pd
import numpy as np

In [2]:
#https://catalogue.data.gov.bc.ca/dataset/bc-schools-k-12-with-francophone-indicators
schools = pd.read_csv("raw/allschools_databc_20201027-fewer-columns.csv")

# Strip whitespace, map school type to match processed rows
schools["PHYSICAL_ADDRESS_CITY"] = schools["PHYSICAL_ADDRESS_CITY"].str.strip()
schools["PUBLIC_OR_INDEPENDENT"] = np.where(schools["PUBLIC_OR_INDEPENDENT"].str.contains("Public School"), "Public", "Independent")


# We are only looking at secondary schools
secondary_schools = schools[schools["SCHOOL_EDUCATION_LEVEL"].str.contains("SECONDARY")]

# Ignore non standard school types
standard_secondary = secondary_schools[secondary_schools["FACILTY_TYPE"].str.contains("Standard School")]

# Remove francophone districts
anglo_standard_secondary = standard_secondary[~standard_secondary["DISTRICT_NAME"].str.contains("Conseil scolaire francophone")]

In [3]:
processed_reports = pd.read_csv("processed_school_reports.csv")
# Building the mapping from one dataset to other, so just look at one year
single_year = processed_reports[processed_reports["YEAR"] == 2019]

# Assuming that if the school name from Fraser is contained in the school name from BC catalogue and the cities and type match -> Same school
merged = anglo_standard_secondary.merge(single_year, left_on=["PHYSICAL_ADDRESS_CITY", "PUBLIC_OR_INDEPENDENT"], right_on=["LOCATION", "SCHOOL_TYPE"])
filtered_merge = merged[merged.apply(lambda x: x.SCHOOL_NAME_y in x.SCHOOL_NAME_x, axis=1)]

# Need to handle duplicates, manual verification from the address
# Code below used to identify multiple schools with same name
# grouped = filtered_merge.groupby("SCHOOL_NAME_y")
# grouped = grouped.filter(lambda x: len(x) > 1)

# Dropping Abbotsford Traditional Senior Secondary 	
filtered_merge = filtered_merge[~filtered_merge["STREET_ADDRESS"].str.contains("2272 Windsor St")]
# Dropping duplicate Langley Fundamental Middle/Secondary Sch
filtered_merge = filtered_merge[~(filtered_merge["STREET_ADDRESS"].str.contains("21250 42 Ave") & (filtered_merge["SCHOOL_NAME_y"] == "Langley"))]
# Dropping Langley Equestrian Academy
filtered_merge = filtered_merge[~filtered_merge["STREET_ADDRESS"].str.contains("20902 37A Ave")]
# Drop duplicate N/S Delta
filtered_merge = filtered_merge[~(filtered_merge["STREET_ADDRESS"].str.contains("11447 82 Ave") & (filtered_merge["SCHOOL_NAME_y"] == "Delta"))]
filtered_merge = filtered_merge[~(filtered_merge["STREET_ADDRESS"].str.contains("750 53 St") & (filtered_merge["SCHOOL_NAME_y"] == "Delta"))]
# Drop Cowichan Open School
filtered_merge = filtered_merge[~filtered_merge["STREET_ADDRESS"].str.contains("1033 Nagle St")]
# Drop St John's Academy Shawnigan Lake
filtered_merge = filtered_merge[~filtered_merge["STREET_ADDRESS"].str.contains("2371 Shawnigan Lake Road")]

grouped = filtered_merge.groupby("SCHOOL_NAME_y")
grouped = grouped.filter(lambda x: len(x) > 1)
grouped

,SCHOOL_YEAR,SCHOOL_NUMBER,SCHOOL_NAME_x,DISTRICT_NUMBER,DISTRICT_NAME,PUBLIC_OR_INDEPENDENT,STREET_ADDRESS,PHYSICAL_ADDRESS_CITY,FACILTY_TYPE,SCHOOL_EDUCATION_LEVEL,...,LOCATION,ENROLMENT,PERCENT_ESL,PERCENT_SPECIAL_NEEDS,PERCENT_FRENCH_IMM,AVERAGE_EXAM_MARK,PERCENT_EXAMS_FAILED,PERCENT_GRAD,PERCENT_DELAYED,OVERALL
5,2019/2020,604015,David Thompson Secondary,6,Rocky Mountain,Public,1535 14th St Unit 1 RR 4,Invermere,Standard School,SECONDARY,...,Invermere,Gr 12 enrolment: 70,0.7,8.8,0.0,67.2,10.5,100.0,16.8,5.9
581,2019/2020,3636246,Princess Margaret Secondary,36,Surrey,Public,12870 72 Ave,Surrey,Standard School,SECONDARY,...,Surrey,Gr 12 enrolment: 278,17.9,10.4,0.0,65.5,10.9,91.9,18.0,4.2
1068,2019/2020,3939018,David Thompson Secondary,39,Vancouver,Public,1755 55th Ave E,Vancouver,Standard School,SECONDARY,...,Vancouver,Gr 12 enrolment: 270,8.4,8.8,0.0,70.0,5.6,96.4,12.6,7.1
1995,2019/2020,6715008,Princess Margaret Secondary,67,Okanagan Skaha,Public,120 Green Ave W,Penticton,Standard School,SECONDARY,...,Penticton,Gr 12 enrolment: 113,0.0,14.1,0.0,69.1,8.1,99.0,3.5,5.5


In [5]:
mapping = filtered_merge[["SCHOOL_NUMBER", "SCHOOL_NAME_y", "LOCATION"]]

associated_number = processed_reports.merge(mapping, left_on=["SCHOOL_NAME", "LOCATION"], right_on=["SCHOOL_NAME_y", "LOCATION"])
associated_number

,SCHOOL_NAME,SCHOOL_TYPE,YEAR,LOCATION,ENROLMENT,PERCENT_ESL,PERCENT_SPECIAL_NEEDS,PERCENT_FRENCH_IMM,AVERAGE_EXAM_MARK,PERCENT_EXAMS_FAILED,PERCENT_GRAD,PERCENT_DELAYED,OVERALL,SCHOOL_NUMBER,SCHOOL_NAME_y
0,Alpha,Public,2015,Burnaby,Gr 12 enrolment: 252,4.7,10.7,12.5,68.3,11.5,94.8,15.0,5.5,4141005,Alpha
1,Alpha,Public,2016,Burnaby,Gr 12 enrolment: 252,4.7,10.7,12.5,67.6,11.7,91.4,13.6,5.3,4141005,Alpha
2,Alpha,Public,2017,Burnaby,Gr 12 enrolment: 252,4.7,10.7,12.5,65.1,16.2,87.7,17.7,3.6,4141005,Alpha
3,Alpha,Public,2018,Burnaby,Gr 12 enrolment: 252,4.7,10.7,12.5,68.7,10.0,95.6,10.2,5.4,4141005,Alpha
4,Alpha,Public,2019,Burnaby,Gr 12 enrolment: 252,4.7,10.7,12.5,68.6,13.4,97.2,6.7,5.2,4141005,Alpha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,Selkirk,Public,2015,Kimberley,Gr 12 enrolment: 68,0.0,10.1,0.0,72.0,4.2,98.6,6.5,7.2,603008,Selkirk
1171,Selkirk,Public,2016,Kimberley,Gr 12 enrolment: 68,0.0,10.1,0.0,70.5,7.7,96.5,5.3,6.8,603008,Selkirk
1172,Selkirk,Public,2017,Kimberley,Gr 12 enrolment: 68,0.0,10.1,0.0,66.3,10.3,96.4,7.2,5.5,603008,Selkirk
1173,Selkirk,Public,2018,Kimberley,Gr 12 enrolment: 68,0.0,10.1,0.0,70.9,7.0,98.7,4.7,6.2,603008,Selkirk


In [6]:
verify_grouping = associated_number.groupby("SCHOOL_NUMBER")
# Each group should have 5 rows --> each corresponding to one year
invalid_groups = verify_grouping.filter(lambda x: len(x) != 5)
invalid_groups

,SCHOOL_NAME,SCHOOL_TYPE,YEAR,LOCATION,ENROLMENT,PERCENT_ESL,PERCENT_SPECIAL_NEEDS,PERCENT_FRENCH_IMM,AVERAGE_EXAM_MARK,PERCENT_EXAMS_FAILED,PERCENT_GRAD,PERCENT_DELAYED,OVERALL,SCHOOL_NUMBER,SCHOOL_NAME_y


In [10]:
# Construct combined table
final_table = anglo_standard_secondary.merge(associated_number, on="SCHOOL_NUMBER").drop("SCHOOL_NAME_y", axis=1).rename(columns={"SCHOOL_NAME_x": "SCHOOL_NAME"})
final_table

,SCHOOL_YEAR,SCHOOL_NUMBER,SCHOOL_NAME,DISTRICT_NUMBER,DISTRICT_NAME,PUBLIC_OR_INDEPENDENT,STREET_ADDRESS,PHYSICAL_ADDRESS_CITY,FACILTY_TYPE,SCHOOL_EDUCATION_LEVEL,...,LOCATION,ENROLMENT,PERCENT_ESL,PERCENT_SPECIAL_NEEDS,PERCENT_FRENCH_IMM,AVERAGE_EXAM_MARK,PERCENT_EXAMS_FAILED,PERCENT_GRAD,PERCENT_DELAYED,OVERALL
0,2019/2020,502001,Mount Baker Secondary,5,Southeast Kootenay,Public,1410 Baker St,Cranbrook,Standard School,SECONDARY,...,Cranbrook,Gr 12 enrolment: 257,0.4,14.4,6.2,66.9,8.9,99.2,8.2,6.8
1,2019/2020,502001,Mount Baker Secondary,5,Southeast Kootenay,Public,1410 Baker St,Cranbrook,Standard School,SECONDARY,...,Cranbrook,Gr 12 enrolment: 257,0.4,14.4,6.2,66.4,10.6,96.8,16.9,6.1
2,2019/2020,502001,Mount Baker Secondary,5,Southeast Kootenay,Public,1410 Baker St,Cranbrook,Standard School,SECONDARY,...,Cranbrook,Gr 12 enrolment: 257,0.4,14.4,6.2,66.2,9.6,99.2,5.1,6.6
3,2019/2020,502001,Mount Baker Secondary,5,Southeast Kootenay,Public,1410 Baker St,Cranbrook,Standard School,SECONDARY,...,Cranbrook,Gr 12 enrolment: 257,0.4,14.4,6.2,68.0,8.7,97.8,9.7,6.7
4,2019/2020,502001,Mount Baker Secondary,5,Southeast Kootenay,Public,1410 Baker St,Cranbrook,Standard School,SECONDARY,...,Cranbrook,Gr 12 enrolment: 257,0.4,14.4,6.2,66.4,8.8,99.6,7.3,6.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,2019/2020,9191024,Nechako Valley Secondary,91,Nechako Lakes,Public,PO Box 950,Vanderhoof,Standard School,ELEMENTARY SECONDARY,...,Vanderhoof,Gr 12 enrolment: 81,0.0,16.6,4.1,66.8,11.1,97.6,9.1,5.8
1171,2019/2020,9191024,Nechako Valley Secondary,91,Nechako Lakes,Public,PO Box 950,Vanderhoof,Standard School,ELEMENTARY SECONDARY,...,Vanderhoof,Gr 12 enrolment: 81,0.0,16.6,4.1,65.6,13.2,96.2,8.3,5.1
1172,2019/2020,9191024,Nechako Valley Secondary,91,Nechako Lakes,Public,PO Box 950,Vanderhoof,Standard School,ELEMENTARY SECONDARY,...,Vanderhoof,Gr 12 enrolment: 81,0.0,16.6,4.1,67.0,12.6,98.0,3.3,6.4
1173,2019/2020,9191024,Nechako Valley Secondary,91,Nechako Lakes,Public,PO Box 950,Vanderhoof,Standard School,ELEMENTARY SECONDARY,...,Vanderhoof,Gr 12 enrolment: 81,0.0,16.6,4.1,71.7,1.6,98.4,7.8,6.9


In [11]:
final_table.to_csv("combined_school_report.csv")